In [10]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import classification_report
import acquire
import prepare
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from io import StringIO
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

# Exercise

### 1. What is your baseline prediction? What is your baseline accuracy? remember: your baseline prediction for a classification problem is predicting the most prevelant class in the training dataset (the mode). When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [11]:
#acquire data
df = acquire.get_titanic_data()

In [12]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,NaN,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,NaN,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,NaN,Southampton,1


In [13]:
df.shape

(891, 13)

In [14]:
#split data
train, validate, test = prepare.prep_titanic_data(df, column = 'age', method = 'median', dummies = ['embarked', 'sex'])


In [15]:
train.shape, validate.shape, test.shape

((498, 12), (214, 12), (179, 12))

In [16]:
train.head(1)

,passenger_id,survived,pclass,age,sibsp,parch,fare,class,alone,embarked_Q,embarked_S,sex_male
583,583,0,1,36.0,0,0,40.125,First,1,0,0,1


In [17]:
# rop out non-numerical columns or non-encoded version remaining in this data set
train.drop(columns = ['class','passenger_id'], inplace=True)
validate.drop(columns = ['class','passenger_id'], inplace=True)
test.drop(columns = ['class','passenger_id'], inplace=True)

In [18]:
train.shape, validate.shape, test.shape

((498, 10), (214, 10), (179, 10))

In [9]:
train.head(2)

,survived,pclass,age,sibsp,parch,fare,alone,embarked_Q,embarked_S,sex_male
583,0,1,36.0,0,0,40.125,1,0,0,1
165,1,3,9.0,0,2,20.525,0,0,1,1


In [ ]:
# We will be attempting to make a Decision Tree Classifier Model that will predict survival on the 
# Titanic that performs better than the baseline

In [21]:
# we are working with survided. The most prevelant class in the training dataset (the mode)
train.survived.value_counts()

0    307
1    191
Name: survived, dtype: int64

In [22]:
train.survived.mode()

0    0
dtype: int64

In [23]:
#so the baselined is survived = 0
train['baseline'] = 0

In [24]:
print(f'The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is {(train.baseline == train.survived).mean():.3}')

The baseline accuracy for nonsurvival in all cases on the Titanic Dataset is 0.616


In [ ]:
# remove baseline assumption from the train, we wont need this column anymore
train.drop(columns='baseline', inplace=True)

In [ ]:

# do the capital X lowercase y thing for train test and split
# X is the data frame of the features, y is a series of the target
X_train, y_train = train[X_cols], train[y_col]
X_validate, y_validate = validate[X_cols], validate[y_col]
X_test, y_test = test[X_cols], test[y_col]

In [41]:
#other way to check the accuracy of baseline
#split 
X_train, y_train = train.drop(columns='survived'), train.survived
#as the baseline mode is = 0 , thats why we put constant = 0
# 1. Create the object
baseline = DummyClassifier(strategy='constant', constant=0)
# 2. Fit the object
baseline.fit(X_train, y_train)

DummyClassifier(constant=0, strategy='constant')

In [42]:
print(f'Baseline accuracy: %.4f' % baseline.score(X_train, y_train))

Baseline accuracy: 0.6165


### 2. Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [27]:
train.head(1)

,survived,pclass,age,sibsp,parch,fare,alone,embarked_Q,embarked_S,sex_male
583,0,1,36.0,0,0,40.125,1,0,0,1


In [30]:
# split our X and y
X_train = train.drop(columns='survived')
y_train = train[['survived']]
X_validate, y_validate = validate[X_cols], validate[y_col]
X_test, y_test = test[X_cols], test[y_col]

In [31]:
# #other way to do the split is decide wich columns we are going to use
# X_cols = [ 'pclass','age', 'sex_male','alone', 'embarked_Q', 'embarked_S']
# y_col = 'survived'

# X_train, y_train = train[X_cols], train[y_col]


In [45]:
# create the model
model = DecisionTreeClassifier()
# fit the model
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [76]:
#calculate the accuracy
acc = model.score(X_train, y_train)
acc

0.9939759036144579

In [77]:
#prediction based on our model
prediction = model.predict(X_train)

In [78]:
#confusion matrix
conf_m = confusion_matrix(y_train, prediction )
conf_m

array([[307,   0],
       [  3, 188]])

In [79]:
#check if rows or columns have the actual number. in this case rows has the actual number
train.survived.value_counts()

0    307
1    191
Name: survived, dtype: int64

In [80]:
 #just to see
mat =  pd.DataFrame ((confusion_matrix(y_train, prediction )),index = ['actual_dead','actual_survived'], columns =['pred_dead','pred_survived' ])
mat

,pred_dead,pred_survived
actual_dead,307,0
actual_survived,3,188


In [81]:
rubric_df = pd.DataFrame([['True Negative', 'False positive'], ['False Negative', 'True Positive']], columns=mat.columns, index=mat.index)
rubric_df

,pred_dead,pred_survived
actual_dead,True Negative,False positive
actual_survived,False Negative,True Positive


In [82]:
rubric_df + ': ' + mat.values.astype(str)

,pred_dead,pred_survived
actual_dead,True Negative: 307,False positive: 0
actual_survived,False Negative: 3,True Positive: 188


positive = survived

these calculate the rates
- tpr = tp/(tp+fn)
- fpr = fp/(fp+tn)
- tnr = tn/(tn+fp)
- fnr = fn/(fn+tp)

https://en.wikipedia.org/wiki/Confusion_matrix

In [84]:
#assign the values
tp = 188
fp =0 
fn=3
tn =307

In [85]:
#calculate the rate
tpr = tp/(tp+fn)
fpr = fp/(fp+tn)
tnr = tn/(tn+fp)
fnr = fn/(fn+tp)

In [ ]:
#     matrix = confusion_matrix(y, model.predict(X))
#     tpr = matrix[1,1] / (matrix[1,1] + matrix[1,0])
#     fpr = matrix[0,1] / (matrix[0,1] + matrix[0,0])
#     tnr = matrix[0,0] / (matrix[0,0] + matrix[0,1])
#     fnr = matrix[1,0] / (matrix[1,1] + matrix[1,0])

In [87]:
print(f'''
The accuracy for our model is {acc:.4}
The True Positive Rate is {tpr:.3}, The False Positive Rate is {fpr:.3},
The True Negative Rate is {tnr:.3}, and the False Negative Rate is {fnr:.3}
''')


The accuracy for our model is 0.994
The True Positive Rate is 0.984, The False Positive Rate is 0.0,
The True Negative Rate is 1.0, and the False Negative Rate is 0.0157



In [56]:
#classification report
clas_rep =pd.DataFrame(classification_report(y_train, prediction, output_dict=True)).T
clas_rep

,precision,recall,f1-score,support
0,0.990323,1.000000,0.995138,307.000000
1,1.000000,0.984293,0.992084,191.000000
accuracy,0.993976,0.993976,0.993976,0.993976
macro avg,0.995161,0.992147,0.993611,498.000000
weighted avg,0.994034,0.993976,0.993967,498.000000


**********************

## other models

In [35]:
model1 = DecisionTreeClassifier(max_depth=3, max_features= 4)
model1.fit(X_train, y_train)
print(f'training score: {model1.score(X_train, y_train):.2%}')


training score: 83.13%


In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model1,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)

In [ ]:
pd.Series(dict(zip(X_train.columns, model1.feature_importances_)))

In [ ]:

model2 = DecisionTreeClassifier(max_depth=3)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
# we want traning score < validate score.
# an aceptable range is a diff of 5%

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model2,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

### 3. Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [ ]:
#prediction based on our model
prediction = model.predict(X_train)

In [ ]:
#model Score
print(f'training score: {model1.score(X_train, y_train):.2%}')
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:
#add my prediction columns to train 
train['prediction'] = model1.predict(X_train)


In [ ]:
train.head()

In [ ]:
#this the numebr of predictions
train.prediction.value_counts()

In [ ]:
#these are the actual numbers
train.survived.value_counts()

In [ ]:
#check if you change the order of labels

In [ ]:
confusion_matrix(train.survived, train.prediction, labels=(1, 0) )

In [ ]:
confusion_matrix(train.survived, train.prediction, labels=(0, 1) )

In [ ]:
#the rows are the actua because 261 + 46 = 307, 
confusion_matrix(train.survived, train.prediction, labels=(0, 1) )

In [ ]:
cm= pd.DataFrame ((confusion_matrix(train.survived, train.prediction, labels=(0, 1))),index = ['actual_death','actual_survived'], columns =['pred_death','pred_survived' ] )

In [ ]:
cm

In [ ]:
pd.DataFrame(classification_report(train.survived, train.prediction,output_dict=True)).T

### 4. Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [ ]:
#### on the validate set *****


positive is : survived
True Positive: predict survived and it is survived
True Negative: predict death and it is death
False Positive: predict survived and it is death
False Negative: predict death and it is survived

In [ ]:
# print the model score on the validate set
print(f'validate score: {model1.score(X_validate, y_validate):.2%}')

In [ ]:
# add model 1 prediction column to the validate df
validate['model1_prediction'] = model1.predict(X_validate)
validate.head()

In [ ]:
# run classification report for model 1 on validate set
model1_class_report = classification_report(validate.survived, validate.model1_prediction, zero_division=True)
print(model1_class_report)

In [ ]:
validate.survived.value_counts()

In [ ]:
confusion_matrix(validate.survived, validate.model1_prediction)

In [ ]:
cm2= pd.DataFrame (confusion_matrix(validate.survived, validate.model1_prediction),index = ['actual_death','actual_survived'], columns =['pred_death','pred_survived' ] )

In [ ]:
cm2

In [ ]:
positive :dead
TP =141
TN = 261
FP = 46
FN = 50


In [ ]:
#accuracy
acc_model1 =accuracy_score(train.survived, train.prediction)

In [ ]:
#precision
prec_model1= precision_score(train.survived, train.prediction, pos_label = 1 )

In [ ]:
#recall
rec_model1= recall_score(train.survived, train.prediction, pos_label = 1 )

In [ ]:
df=pd.DataFrame(classification_report(train.survived, train.prediction,output_dict=True))

In [ ]:
df

In [ ]:
df.rename(columns={'0': "dead", '1': "survived"})

### 5. Run through steps 2-4 using a different max_depth value.

In [ ]:

model2 = DecisionTreeClassifier(max_depth=3)
model2.fit(X_train, y_train)

print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(24, 12))
plot_tree(
    model2,
    feature_names=X_train.columns.tolist(),
    class_names=['died', 'survived'],
)

In [ ]:
pd.Series(dict(zip(X_train.columns, model2.feature_importances_)))

In [ ]:
#model Score
print(f'training score: {model2.score(X_train, y_train):.2%}')
print(f'validate score: {model2.score(X_validate, y_validate):.2%}')

In [ ]:
#add my prediction columns to train 
train['prediction_2'] = model2.predict(X_train)


In [ ]:
train.head()

In [ ]:
#these are the actual numbers
train.survived.value_counts()

In [ ]:
#these are the prediction_2 numbers
train.prediction_2.value_counts()

In [ ]:
#the rows are the actua because 261 + 46 = 307, 
confusion_matrix(train.survived, train.prediction_2, labels=(0, 1) )

In [ ]:
cm2= pd.DataFrame ((confusion_matrix(train.survived, train.prediction_2, labels=(0, 1))),index = ['actual_death','actual_survived'], columns =['pred_death','pred_survived' ] )

In [ ]:
cm2

In [ ]:
df2=pd.DataFrame(classification_report(train.survived, train.prediction_2,output_dict=True))
df2

In [ ]:
#accuracy
acc_model2 =accuracy_score(train.survived, train.prediction_2)

In [ ]:
#precision
prec_model2 =precision_score(train.survived, train.prediction_2, pos_label = 1 )

In [ ]:
#recall
rec_model2= recall_score(train.survived, train.prediction_2, pos_label = 1 )

### 6. Which model performs better on your in-sample data?

In [ ]:
acc_model1, prec_model1, rec_model1

In [ ]:
acc_model2, prec_model2, rec_model2


### 7.Which model performs best on your out-of-sample data, the validate set?

In [ ]:
validate.shape

In [ ]:
train.shape

In [ ]:
#accuracy
acc_model2 =accuracy_score(validate.survived, train.prediction_2)

In [ ]:
#precision
prec_model2 =precision_score(validate.survived, train.prediction_2, pos_label = 1 )

In [ ]:
#recall
rec_model2= recall_score(validate.survived, train.prediction_2, pos_label = 1 )

# Random Forest

In [ ]:
Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 10.

Evaluate your results using the model score, confusion matrix, and classification report.

Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

Run through steps increasing your min_samples_leaf and decreasing your max_depth.

What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

After making a few models, which one has the best performance (or closest metrics) on both train and validate?